In [4]:
DATASET = "swedish_tunes.csv"
TRANSLATION_FILE = "translation"
INPUT_SIZE = 5
HIDDEN_LAYERS = 0
NUM_LAYERS = 0

In [ ]:
import pandas
from torch.nn import RNN, Linear, LogSoftmax, Module
import torch
import argparse
import math

In [ ]:
# Class that creates the model that we'll use
class PredictRNN(Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PredictRNN, self).__init__()

        self.rnn = RNN(input_size, hidden_size)
        self.h2o = Linear(hidden_size, output_size)
        self.softmax = LogSoftmax(dim=1)

    def forward(self, line_tensor):
        rnn_out, hidden = self.rnn(line_tensor)
        output = self.h2o(hidden[0])
        output = self.softmax(output)
        return output

In [ ]:
# Performs one-hot encoding on a single token vector
def onehot(line, n_tokens, translation):
    tensor = torch.zeros(len(line), 1, n_tokens)
    for ind, token in enumerate(line):
        tensor[ind][0][translation.find(token)] = 1
    return tensor


In [8]:
# Get min number of tokens in a line
def min_line_size(filepath):
    file_p = open(filepath, 'r', encoding='latin-1')
    min_tokens = float('inf')
    for line in file_p.readlines():
        line_tok = line.split(',')
        line_len = len(line_tok)
        if line_len < min_tokens:
            min_tokens = line_len
    file_p.close()
    return min_tokens


In [ ]:
# calculates the accuracy of the predictions of a neural network
# For classification tasks
# NOTE: X and y should already be PyTorch tensors
def calculate_accuracy(network, X, y):
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        X = X.to(device)
        y = y.to(device)
    # make predictions for the given X
    probs = network(X)
    predictions = torch.argmax(probs, dim=1)
    # the calculate accuracy of those predictions
    total = len(predictions)
    accuracy = sum(predictions == y) / total
    variance = accuracy * (1 - accuracy)
    std_err = math.sqrt(variance / total)
    up_bound = accuracy + 2.39 * std_err
    low_bound = accuracy - 2.39 * std_err
    return float(accuracy), up_bound, low_bound


In [ ]:
# converts a training set into smaller train and validation sets
def create_validation(training_X, training_y, valid_percentage):
    # find the split point between training and validation
    training_n = training_X.shape[0]
    valid_rows = int(valid_percentage * training_n)

    # create the validation set
    valid_X = training_X.iloc[:valid_rows]
    valid_y = training_y.iloc[:valid_rows]

    # create the (smaller) training set
    train_X = training_X.iloc[valid_rows:]
    train_y = training_y.iloc[valid_rows:]

    return train_X, train_y, valid_X, valid_y


In [ ]:
# trains a neural network with given training data
def train_network(network, training_X, training_y, lr, is_classification, verbose=False):
    # split the training data into train and validation
    # Note: use 20% of the original training data for validation
    train_X, train_y, valid_X, valid_y = create_validation(training_X, training_y, 0.2)

    # convert our data to PyTorch objects
    train_X = torch.from_numpy(train_X.values).long()
    train_y = torch.from_numpy(train_y.values).long()
    valid_X = torch.from_numpy(valid_X.values).long()
    valid_y = torch.from_numpy(valid_y.values).long()

    # move the data and model to the GPU if possible
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print(f"Using device {torch.cuda.get_device_name(device)}")
        train_X = train_X.to(device)
        train_y = train_y.to(device)
        valid_X = valid_X.to(device)
        valid_y = valid_y.to(device)
        network = network.to(device)

    # create the algorithm that learns the weight for the network
    optimizer = torch.optim.Adam(network.parameters(), lr=lr)

    # create the loss function that tells optimizer how much error it has in its predictions
    # here we use cross entropy since we have a classification task with more than two possible labels
    loss_function = torch.nn.CrossEntropyLoss()

    # train for 1000 epochs
    num_epochs = 1000
    train_loss_values = []
    valid_loss_values = []
    train_acc_values = []
    valid_acc_values = []
    for epoch in range(num_epochs):
        # make predictions on the training set and validation set
        train_predictions = network(train_X)
        valid_predictions = network(valid_X)
        train_loss = loss_function(train_predictions, train_y)

        # calculate the error on the training set
        train_loss_values.append(train_loss.item())
        valid_loss_values.append(loss_function(valid_predictions, valid_y).item())
        train_acc_values.append(calculate_accuracy(network, train_X, train_y))
        valid_acc = calculate_accuracy(network, valid_X, valid_y)
        valid_acc_values.append(valid_acc)

        # Early return for perfect fit
        if valid_acc == 1:
            # convert the training progress data to a Pandas DataFrame
            progress = {
                "epoch": range(epoch+1),
                "train_loss": train_loss_values,
                "valid_loss": valid_loss_values,
                "train_acc": train_acc_values,
                "valid_acc": valid_acc_values
            }
            return pandas.DataFrame(progress)

        # perform backpropagation
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    # convert the training progress data to a Pandas DataFrame
    progress = {
        "epoch": range(num_epochs),
        "train_loss": train_loss_values,
        "valid_loss": valid_loss_values,
        "train_acc": train_acc_values,
        "valid_acc": valid_acc_values
    }

    return pandas.DataFrame(progress)


In [2]:
def process_dataset(dataset_filepath, input_size):
    max_size = min_line_size(dataset_filepath)
    if input_size + 3 > max_size:
        raise ValueError(f"Input size {input_size} greater than maximum allowed size {max_size}")

    # We need to create a dataframe with the proper number of columns
    # Get the first three tokens from every line and store them in front of each instance
    dataframe_dict = {
        'timesig': [],
        'key': [],
        'style': []
    }

    for i in range(input_size):
        dataframe_dict[i] = []

    dataset_fp = open(dataset_filepath, 'r')
    for line in dataset_fp.readlines():
        line = line.split(',')
        timesig = line[0]
        key = line[1]
        style = line[2]
        remainder = line[3:]
        for i in range(len(remainder) - input_size):
            window = remainder[i:i+input_size]
            dataframe_dict['timesig'].append(timesig)
            dataframe_dict['key'].append(key)
            dataframe_dict['style'].append(style)
            for ind, token in enumerate(window):
                dataframe_dict[ind].append(token)

    dataset_fp.close()
    return pandas.DataFrame(dataframe_dict)



In [ ]:
dataset_fp = open(DATASET, 'r')
translation_fp = open(TRANSLATION_FILE, 'r')

# Load translation into indexable array
translation = []
for line in translation_fp.readlines():
    translation.append(line)
num_tokens = len(translation)



# Now split each line of input into all possible slid windows and add them

dataset_fp.close()
translation_fp.close()

In [9]:
process_dataset(DATASET, INPUT_SIZE)

ValueError: Input size 5 greater than maximum allowed size 2